In [ ]:
plt.figure(figsize=(20,10))
cor = df.corr()
mask = np.triu(np.ones_like(df.corr(), dtype=bool))
sns.heatmap(cor, annot=True, cmap='BrBG', mask=mask, vmin=-1, vmax=1)#cmap=plt.cm.Reds)
plt.show()

In [ ]:
cor2 = cor.where(mask).round(2)
cor2 = cor2.stack().reset_index().rename({0: "Correlation"}, axis=1)
cor2 = cor2[~(cor2["level_0"]==cor2["level_1"])]
cor2["Abs. Corr."] = abs(cor2["Correlation"])
cor2 = cor2.sort_values("Abs. Corr.", ascending=False)

display(cor2.head(10))
display(cor2.tail(10))

In [ ]:
def VIF(X, threshold = 5):
  from statsmodels.stats.outliers_influence import variance_inflation_factor
  columns = X.columns.to_list()
  vif = pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i)\
                       for i in range(len(columns))]
  vif["features"] = X.columns

  ## Check for infinity VIF
  mask = (vif["VIF Factor"] == np.infty)
  if mask.any():
    to_drop=vif[mask]["features"].to_list()
    setColumns = set(columns)
    setTo_drop = set(to_drop)
    columns_new = list(setColumns.difference(setTo_drop))
    print(f"dropping due to infinite VIF:")
    print("\n".join(setTo_drop))
    return VIF(X[columns_new], threshold=threshold)

  # Drop feature with highest VIF if exceeding threshold
  if vif["VIF Factor"].max() > threshold:
    index = vif["VIF Factor"].idxmax()
    to_drop = vif.loc[index].to_dict()
    print("dropping {features} at index {index} with VIF of {VIF Factor:.2f} exceeding threshold of {threshold}"\
          .format(**to_drop, index=index, threshold=threshold))
    columns.remove(to_drop["features"])
    return VIF(X[columns], threshold)
  
  return vif

In [ ]:
vif = VIF(df_scaled, 10).round(2)
vif